## 4.1 누락된 데이터 다루기

### 4.1.1 테이블 형태 데이터에서 누락된 값 식별

In [8]:
import pandas as pd
from io import StringIO

csv_data = \
"""A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
"""
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0


In [2]:
# null값 확인
df.isnull().sum()

A    0
B    0
C    1
D    0
dtype: int64

### 4.1.2 누락된 값이 있는 훈련 샘플이나 특성 제외

In [3]:
df.dropna(axis=0) # axis=0 은 행 삭제, axis=1은 열 삭제

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [4]:
# 모든 열이 NaN일 때만 행을 삭제
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0


In [5]:
# NaN이 아닌 값이 4개보다 작은 행을 삭제
df.dropna(thresh=4) 

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [6]:
# 특정 열에 NaN이 있는 행만 삭제
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0


### 4.1.3 누락된 값 대체 
- 보간(interpolation) 기법

In [9]:
# 평균으로 대체하는 보간 기법
# 각 특성 열의 전체 평균으로 누락된 값 대체
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[1., 2., 3., 4.],
       [5., 6., 3., 8.]])

위의 결과에서 1,2,4번째 열의 수치 총합 -> 1+5+2+6+4+8 = 24

각 열의 평균 24 / 4 = 6

-> 3번째 열의 누락값 3으로 대체